#### Dependencies

In [1]:
!pip install tensorflow tensorflow-cpu opencv-python matplotlib

In [2]:
!pip list

Package                       Version
----------------------------- ---------------
absl-py                       1.3.0
aiohttp                       3.8.3
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.11.1
anaconda-navigator            2.4.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.w

#### Importing Tensorflow and lessening GPU consumption

In [3]:
import tensorflow as tf
import os

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[]

#### Importing Opencv and removing dogy images

In [6]:
import cv2
import imghdr

In [7]:
data_dir = 'data' 

In [8]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [9]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

#### Loading Training data

In [10]:
import numpy as np

In [11]:
image_paths = []
labels = []
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        image_paths.append(image_path)
        labels.append(image_class)

In [12]:
# Convert labels to numeric format
label_mapping = {label: i for i, label in enumerate(set(labels))}
labels = [label_mapping[label] for label in labels]

In [13]:
# Convert image paths to numpy array
image_paths = np.array(image_paths)
labels = np.array(labels)

#### Perform k-fold cross-validation

In [ ]:
!pip install scikit-learn
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_accuracy = []

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for fold, (train_indices, test_indices) in enumerate(skf.split(image_paths, labels), 1):
    print(f'Fold {fold}/{k}')

    train_paths = image_paths[train_indices]
    train_labels = labels[train_indices]
    test_paths = image_paths[test_indices]
    test_labels = labels[test_indices]

#### Load and preprocess images

In [ ]:

train_images = []
for path in train_paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))
    img = img / 255.0
train_images.append(img)
train_images = np.array(train_images)

In [ ]:
test_images = []
for path in test_paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))
    img = img / 255.0
test_images.append(img)
test_images = np.array(test_images)

#### Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

#### Train

In [ ]:
model.fit(train_images, train_labels, epochs=20, batch_size=32)

#### Evaluate

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
fold_accuracy.append(accuracy)

In [ ]:
print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('Binary Accuracy:', binary_accuracy)

In [ ]:
mean_accuracy = np.mean(fold_accuracy)
print('Average Accuracy:', mean_accuracy)

#### Testing

In [ ]:
import cv2

In [ ]:
img = cv2.imread('Productivity.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Good')
else:
    print(f'Predicted class is Bad')

#### Saving the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))